In [1]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import random
import pandas as pd
import re
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="o3-mini")
# llm = ChatDeepSeek(
#     model="deepseek-reasoner",
#     temperature=0.0,
# )

```RAG Tool Functions -> Function Caller -> Metrics Recommendor -> Metrics Calculator -> Analyzer -> Executor```

In [3]:
master_prompt = """

You are a DeFi Agent Launcher for Xade AI: A meta-agent that creates agents for users that can fetch, analyze DeFi data as well as execute actions based on them for the data. 
Your output is always plaintext, without any markdown.

---

You will receive the following input:

Agent Name & Description:
// The name of the agent and a short description of what it does

Query:
// A very simple, elementary query about the user's requirements which you will have to expand on and detail
 
Actions:
// Any actions the agent may be required to perform
# Eg.
#   Sentient Posting:
#   - Posting on X and XADE Terminal 
#   - Every 'x' minutes
#   - Reply on tweets by Elon Musk (@ElonMusk)
#   - Reply to mentions and quotes of oneself (@AgentName)

Specific Data Sources to Use:
// Any specific data sources to use (Separate from your recommendations)

Style:
// The style of the response the user would like to receive

Examples:
// Example interactions

---

INSTRUCTIONS:

Your task is to review the challenge, instruct and allocate tasks to your underlying LLM agents to obtain the relevant data, synthesize it, analyze it, customize responses and execute some actions.

---

You will have access to agents to automate the task. The workflow for your agents is as follows:

Data Functions Retriever → DeFi Data Fetcher & Metrics Recommender → Data Processor & Metrics Calculator → DeFi Analyzer → DeFi Executor


- Data Functions Retriever: A RAG model that can retrieve functions to fetch data relevant to the task from various sources.
  Currently, it has access to three data providers:
    - Mobula: A provider for market-based data in DeFi
    - LunarCrush: A provider for social/trending metrics in DeFi
    - CryptoPanic: A provider for news and social media posts in DeFi
You must craft the input for the RAG model keeping in mind that it can only retrieve functions to get the data, and cannot directly fetch the data or apply any filters.
If you need to identify any specific tokens on a specific blockchain or from a specific sector of DeFi (for e.g. ReFi), ask the RAG model for the functions to get those tokens.
Directly mention the kind of data you need (sentiment-related, volatility-related) or any category-specific data.
Any filters you want for the data from the RAG model should be mentioned in the instructions for the DeFi Fetcher, and not in the RAG model input

- DeFi Data Fetcher & Metrics Recommender: An agent that can call the functions retrieved by the Function Retriever to fetch data from various sources, and recommend metrics to calculate based off of that data.
  This agent will call the data functions retrieved by the Functions Retriever and organize the data for Processor. Additionally, it will also instruct the Processor on what metrics to calculate based on the data and how.
You must provide the Fetcher with a detailed description of the user's requirements that can assist it with it's recommendations for the Processor.
(NOTE: IT CANNOT YET ACCESS TWEET/X DATA)

- A DeFi Analyzer: An agent that can analyze DeFi data and provide insights based on the data. 
This agent generates a data-driven analysis and evaluation of the data while using suitable methodologies (STARE, CAR , SOAR, IDEA frameworks). 
It is smart enough to understand the context of the data and provide insights based on the data.
Based on your instructions and the data given by the Data Fetcher, it can model its analysis to:
  - Discover emerging trends
  - Analyze technical indicators such as RSI, MACD, and SMA to predict movements
  - Interpret social media metrics to gauge market sentiment and mindshare
Thus, ensure you give instructions such that the analyzer can provide insights relevant to the user's query.


- A DeFi Executor: An agent that can execute actions based on the data fetched and analyzed by the other two agents. 
This agent can devise trading strategies, execute trades, post on X and the XADE Terminal every few minutes, reply to various pre-defined indiviuals, join Twitter spaces, perform portoflio analysis, and other such actions.
It's execution will be performed by calling the relevant functions.
It will execute the actions you ask it to, based on the data fetched and analyzed by the other two agents.
It will have access to a memory of it's tasks, so it can remember its previous context and actions and adjust future actions accordingly.
Here is a list of it's abilities:
  - Post on X and the XADE Terminal periodically
  - Reply to posts by various pre-defined individuals
  - Join Twitter spaces and engage in discussions
  - Perform portfolio analysis for a given wallet address
  - Devise trading strategies based on the user's priorities and the analyzed data.
  - For text-based actions, it can customize responses based on the user's preferences.

---

When creating the instructions for the LLMs to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - Primary actions must be clearly defined. You must define the end-goal that the LLM is trying to achieve through its actions.
    - Sub-actions must be indented and clearly defined, beginning on a new line.
    - Specify conditions using clear 'if...then...else' statements
    - Ensure that there is no ambiguity in any part of your plan. You must be specific and clear about what to use and how.
    - The instructions generated must be extremely detailed and thorough with explanations at every step. They must first outline the primary instruction for the LLM and then provide direct and thorough explanations

NO MARKDOWN IS ALLOWED.    
---

FORMAT INSTRUCTIONS:

{format_instructions}

---

{master_example}

TASK:
{input}

"""

In [4]:
master_example = '''

TASK:
Agent Name & Description:
Agent DAO
Performs detailed analysis on trending DAOs and provides insights on various aspects.

Query:
Comment on rising DAOs

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply to mentions and quotes of oneself (@AgentDAO)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like an Analyst

Examples:
N.A.

INSTRUCTIONS:

1. Data Functions Retriever Input:
Query: 
Retrieve functions to
- Identify Top DAOs
- Extract information about those DAOs
- Extract news about those DAOs

2. DeFi Data Fetcher & Metrics Recommender Instructions:
Action: Fetch and filter DAO data from provided sources
- Call functions provided to fetch the required data
- Filter top 10 DAOs based on trading volume. (or closest available parameter)
- Collect information about these DAOs and organize them
- Fetch news about these DAOs and sort latest 20 items
- Recommend metrics to calculate based on fetched data
  User requirements: A detailed analysis of DAOs and their performance

3. Data Processor & Metrics Calculator Instructions:
Follow DeFi Data Fetcher's instructions to calculate metrics

4. DeFi Analyzer Instructions:
Action: Perform detailed analysis with strong growth signals using the STARE framework (Scan, Target, Analyze, Respond, Evaluate).  
  - Scan: Cross-reference information provided across data sources.  
  - Target: Rank DAOs using the metrics provided.
  - Analyze:  
    - Identify correlations in the provided data and metrics.
    - Flag key points about provided DAOs for Evaluation Report.
  - Respond:
    - Provide a detailed, structured report on provided DAOs 
    - Provide key, actionable insights as well as unique observations about data
  - Evaluate:  
    - Consider the rise in DAOs and create comments
    - Perform aspect-based analysis and evaluation of each DAO and the entire category

5. DeFi Executor Instructions:
- Primary Action: Post analysis on X/XADE Terminal every 5 minutes and reply to @AgentETH mentions.  
  - Posting Workflow:  
    - Content Structure:  
      - Header: "Rising DAO Alert: [DAO Name]".  
      - Body: "Up [X]% in trading volume + [Y]% social growth. Recent news: [Summary]."  
      - Footer: "Metrics: Trend Score [Z]/10 | Sentiment: [A]%".  
    - Priority System:  
      - Post High-Priority DAOs immediately.  
      - Post Medium-Priority DAOs if no High-Priority candidates exist.  
  - Reply to Mentions:  
    - If user asks, "Why is [DAO] trending?":  
      - Respond with: "Based on [metric] growth and [news headline], [DAO] is gaining traction due to [reason]."  
    - If user requests further analysis/comparison:  
      - Deploy data-processor-analyzer pipeline to compute and provide reply.

    Example Output for X Post  
    ```
    Rising DAO Alert: @MakerDAO  
    - Trading volume +47% in 24h  
    - Social mentions +62% (92% positive)  
    - Recent news: MakerDAO launches "Endgame" restructuring plan.  
    Metrics: Trend Score 9.2/10 | Sentiment: 92%  
    ```

'''


In [5]:
from typing import Optional

from pydantic import BaseModel, Field

class ElPlan(BaseModel):
    rag: str = Field(description="The task for the RAG model and the instructions for execution.")
    fetcher: str = Field(description="The task for the DeFi Data Fetcher and the insturctions for execution.")
    analyzer: str = Field(description="The task for the analyzer and the instructions for execution (detailed). This task must be possible to fulfill with data retreived by the Data Fetcher.")
    executor: str = Field(description="The task for the executor and the instructions to fulfill it. All specific instructions for each action can be included here as well. All miscellaneous instructions can be included here.")

parser = PydanticOutputParser(pydantic_object=ElPlan)

In [6]:
master_prompt = PromptTemplate(
    template=master_prompt,
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [7]:
input = """
Agent Name & Description:
Agent ETH
Performs detailed analysis on top Ethereum Ecosystem Projects and provides insights on various aspects.

Query:
Comment on top ETH ecosystem projects

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply to mentions and quotes of oneself (@AgentETH)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like a DeGen

Examples:
N.A.

INSTRUCTIONS:
"""

response = llm.invoke(master_prompt.format(input=input, master_example=master_example))

In [8]:
print(response.content)

{
  "rag": "Instruct the RAG model to retrieve functions to: • Identify top Ethereum ecosystem projects from the data sources; • Extract detailed market data, social metrics, and news about these projects from Market Data, Social Data, and News Feeds; • Specifically, look for functions that can filter the projects based on high trading volumes and social engagement indicating project traction.",
  "fetcher": "Direct the DeFi Data Fetcher & Metrics Recommender to fetch Ethereum ecosystem projects data using the functions retrieved. Steps: • Call the functions to fetch market data and social sentiment for Ethereum ecosystem projects. • Filter and rank the top 10 projects using trading volume or a similar parameter if available. • Organize detailed information for each project including metrics like trading volume, pace of social growth, and any trending indicators. • Fetch up to the latest 20 news items related to these projects from the News Feeds. • Recommend metrics for calculating tr

In [9]:
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
import json

def display_agent_output(response_content):
    console = Console()
    response_content = response_content.strip('```json\n').strip('\n```')
    try:
        data = json.loads(response_content)
    except:
        console.print("[bold red]Error parsing response as JSON[/bold red]")
        return
    
    table = Table(show_header=True, header_style="bold")
    table.add_column("Component", style="italic white")
    table.add_column("Instructions", style="")
    
    for key, value in data.items():
        formatted_key = key.replace('_', ' ').title()
        table.add_row(formatted_key, value)
    
    console.print(Panel(
        table,
        title="[bold yellow]🤖 DeFi Agent Instructions[/bold yellow]",
        border_style="yellow"
    ))

In [10]:
display_agent_output(response.content)

╭────────────────────────────────────────── 🤖 DeFi Agent Instructions ───────────────────────────────────────────╮
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Component ┃ Instructions                                                                                    ┃ │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ Rag       │ Instruct the RAG model to retrieve functions to: • Identify top Ethereum ecosystem projects     │ │
│ │           │ from the data sources; • Extract detailed market data, social metrics, and news about these     │ │
│ │           │ projects from Market Data, Social Data, and News Feeds; • Specifically, look for functions that │ │
│ │           │ can filter the projects based on high trading volumes and social engagement indicating project  │ │
│ │           │ traction.                                                                                       │ │
│ │ Fetcher   │ Direct the DeFi Data Fetcher & Metrics Recommender to fetch Ethereum ecosystem projects data    │ │
│ │           │ using the functions retrieved. Steps: • Call the functions to fetch market data and social      │ │
│ │           │ sentiment for Ethereum ecosystem projects. • Filter and rank the top 10 projects using trading  │ │
│ │           │ volume or a similar parameter if available. • Organize detailed information for each project    │ │
│ │           │ including metrics like trading volume, pace of social growth, and any trending indicators. •    │ │
│ │           │ Fetch up to the latest 20 news items related to these projects from the News Feeds. • Recommend │ │
│ │           │ metrics for calculating trend scores, social sentiment percentages, and growth signals tailored │ │
│ │           │ to Ethereum projects, in preparation for detailed analysis.                                     │ │
│ │ Analyzer  │ Instruct the DeFi Analyzer to perform a detailed analysis on the fetched Ethereum ecosystem     │ │
│ │           │ projects using the STARE framework, while adopting a DeGen tone: • SCAN: Cross-reference market │ │
│ │           │ data, social metrics, and news for consistency and trends. • TARGET: Rank the projects based on │ │
│ │           │ the recommended metrics such as trading volume increase, social growth, and news impact. •      │ │
│ │           │ ANALYZE: Identify correlations, highlight standout metrics, and flag key trends; calculate      │ │
│ │           │ technical and sentiment indicators including trend scores and hype levels, and evaluate each    │ │
│ │           │ project aggressively using risk-reward angles typical in DeGen parlance. • RESPOND: Generate a  │ │
│ │           │ detailed yet edgy report with segments on each project, highlighting results with percentages   │ │
│ │           │ (e.g., trading volume +X%, social mentions Y%, etc.) and mention eye-catching news headlines. • │ │
│ │           │ EVALUATE: Provide commentary on why these projects are trending, potential breakout signals,    │ │
│ │           │ and implications for market movements. The analysis should be structured, actionable, and       │ │
│ │           │ display a no-nonsense DeGen attitude.                                                           │ │
│ │ Executor  │ Instruct the DeFi Executor to carry out sentient posting on both X and XADE Terminal every 5    │ │
│ │           │ minutes, and to reply to mentions and quotes of @AgentETH. Posting instructions: • For each     │ │
│ │           │ high-ranked or breakout project, generate a post with the following structure: Header: 'Top ETH │ │
│ │           │ Alert: [Project Name]'; Body: a brief summary including metrics such as trading volume          │ │
│ │           │ increase, social growth percentage, and a headline summary from the latest news; Footer:        │ │
│ │           │ 'Score: [Trend Score]/10 | Sentiment: [Se

In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import inspect
from mobula import *
from social import *


def build_knowledge_base():
    functions = [
      Mobula.get_all_assets,
      Mobula.get_blockchains,
      Mobula.get_market_blockchain_pairs,
      Mobula.get_market_blockchain_stats,
      Mobula.get_cefi_funding_rate,
      Mobula.get_feed_create,
      Mobula.get_market_history_pair,
      Mobula.get_market_history,
      Mobula.get_market_multi_data,
      Mobula.get_market_multi_history,
      Mobula.get_market_nft,
      Mobula.get_market_pair,
      Mobula.get_market_pairs,
      Mobula.get_market_query_token,
      Mobula.get_wallet_nfts,
      Mobula.get_wallet_transactions,
      Mobula.get_wallet_history,
      Mobula.get_wallet_multi_portfolio,
      Mobula.get_metadata,
      Mobula.get_multi_metadata,
      Mobula.get_metadata_categories,
      Mobula.get_metadata_news,
      Mobula.get_metadata_trendings,
      Mobula.get_market_query,
      Mobula.get_market_sparkline,
      Mobula.get_market_token_holders,
      Mobula.get_market_token_vs_market,
      Mobula.get_market_total,
      Mobula.search,
      Mobula.get_market_data,
      Mobula.get_wallet_portfolio,
      Mobula.get_blockchain_pairs,
      LunarCrush.get_coin_data, 
      LunarCrush.get_coin_metadata, 
      LunarCrush.get_nft_data, 
      LunarCrush.get_topic_news, 
      LunarCrush.get_coins_list, 
      LunarCrush.get_topic_summary, 
      CryptoPanic.get_posts
    ]
    kb = {}
    for func in functions:
        source = inspect.getsource(func)
        doc = func.__doc__
        kb[func.__name__] = {"description": doc, "code": source}
    return kb

KNOWLEDGE_BASE = build_knowledge_base()


In [14]:
# ----------------------------
# Build the Vector Database using FAISS and SentenceTransformer
# ----------------------------

# Prepare the list of function names and corresponding descriptions from the knowledge base.
function_names = list(KNOWLEDGE_BASE.keys())
function_descriptions = [KNOWLEDGE_BASE[name]["description"] for name in function_names]

In [15]:
function_names,function_descriptions

(['get_all_assets',
  'get_blockchains',
  'get_market_blockchain_pairs',
  'get_market_blockchain_stats',
  'get_cefi_funding_rate',
  'get_feed_create',
  'get_market_history_pair',
  'get_market_history',
  'get_market_multi_data',
  'get_market_multi_history',
  'get_market_nft',
  'get_market_pair',
  'get_market_pairs',
  'get_market_query_token',
  'get_wallet_nfts',
  'get_wallet_transactions',
  'get_wallet_history',
  'get_wallet_multi_portfolio',
  'get_metadata',
  'get_multi_metadata',
  'get_metadata_categories',
  'get_metadata_news',
  'get_metadata_trendings',
  'get_market_query',
  'get_market_sparkline',
  'get_market_token_holders',
  'get_market_token_vs_market',
  'get_market_total',
  'search',
  'get_market_data',
  'get_wallet_portfolio',
  'get_blockchain_pairs',
  'get_coin_data',
  'get_coin_metadata',
  'get_nft_data',
  'get_topic_news',
  'get_coins_list',
  'get_topic_summary',
  'get_posts'],
 ['\n        Get data for all assets.\n\n        Args:\n    

In [16]:
import json

json_data = json.loads(response.content)
rag_instructions = json_data["rag"]
print(rag_instructions)

Instruct the RAG model to retrieve functions to: • Identify top Ethereum ecosystem projects from the data sources; • Extract detailed market data, social metrics, and news about these projects from Market Data, Social Data, and News Feeds; • Specifically, look for functions that can filter the projects based on high trading volumes and social engagement indicating project traction.


In [24]:
from typing import List, Dict
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

def process_rag_query(
    rag_instructions: str,
    function_names: List[str],
    function_descriptions: List[str]
) -> Dict[str, Dict[str, str]]:
    """
    Process RAG instructions to find relevant functions using embeddings and FAISS.
    
    Args:
        rag_instructions (str): The instructions to process.
        function_names (List[str]): List of available function names.
        function_descriptions (List[str]): List of function descriptions corresponding to function_names.
        
    Returns:
        Dict[str, Dict[str, str]]: Dictionary mapping function names to their descriptions and code.
    """
    # Ensure the input lists are consistent.
    if len(function_names) != len(function_descriptions):
        raise ValueError("The number of function names must match the number of function descriptions.")
    
    # Generate an optimized RAG query using an LLM.
    # Make sure that ChatOpenAI is imported and configured correctly in your environment.
    llm = ChatOpenAI(model="gpt-4o-mini")
    messages = [
        {"role": "system", "content": "You are a DeFi data specialist that converts instructions into clear, concise search queries focused on finding relevant data functions."},
        {"role": "user", "content": f"Convert these instructions into a focused search query optimized for finding relevant DeFi data functions: {rag_instructions}"}
    ]
    completion = llm.invoke(messages)
    rag_query = completion.content.strip()
    
    if not rag_query:
        raise ValueError("Generated RAG query is empty.")
    
    # Initialize the SentenceTransformer embedding model.
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Compute the embedding for the generated query.
    query_embedding = embedding_model.encode(rag_query)
    
    # Compute embeddings for each function description.
    description_embeddings = [embedding_model.encode(desc) for desc in function_descriptions]
    
    # Convert embeddings to numpy arrays for FAISS.
    query_embedding_np = np.array(query_embedding).reshape(1, -1).astype('float32')
    description_embeddings_np = np.array(description_embeddings).astype('float32')
    
    # Create a FAISS index using L2 distance metric.
    dimension = query_embedding_np.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(description_embeddings_np)
    
    # Search for the most similar functions (top 3, or fewer if not available).
    k = min(3, len(function_descriptions))
    distances, indices = index.search(query_embedding_np, k)
    
    # Build the result dictionary with matching function names and their descriptions.
    results = {}
    for idx in indices[0]:
        func_name = function_names[idx]
        results[func_name] = {
            "description": function_descriptions[idx],
            "code": ""  # Placeholder for code
        }
    
    return results

In [25]:
results = process_rag_query(rag_instructions=rag_instructions, function_descriptions=function_descriptions, function_names=function_names)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Initialize the embedding model (using a lightweight model here; adjust as needed).
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")



In [ ]:
print("hello world")

In [ ]:
# Compute embeddings for each function description.
embeddings = embedding_model.encode(function_descriptions)
embeddings = np.array(embeddings).astype("float32")

In [ ]:
dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dim)
faiss_index.add(embeddings)

In [ ]:
def rag_search_vector(query: str, top_k: int = 3) -> dict:
    """
    RAG Search with a vector database.
    Given a query string, this function computes its embedding and retrieves the top_k
    function entries whose description embeddings are closest.
    
    Inputs:
        - query: A string query.
        - top_k: Number of top results to return.
    Output:
        - A dictionary mapping function names to their 'description' and 'code'.
    """
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")
    
    distances, indices = faiss_index.search(query_embedding, top_k)
    results = {}
    for idx in indices[0]:
        func_name = function_names[idx]
        results[func_name] = KNOWLEDGE_BASE[func_name]
    return results


In [ ]:
sample_query = "Retrieve functions to get the Simple Moving Average (SMA) of a price history."

matching_functions = rag_search_vector(sample_query, top_k=3)

if matching_functions:
  print("Matching Functions for query:", sample_query)
  for name, details in matching_functions.items():
      print(f"\nFunction: {name}\nDescription:\n{details['description']}\nCode:\n{details['code']}\n{'-'*40}")
else:
  print("No matching functions found.")